In [1]:
import pandas as pd
import json
import time
from geopy.exc import GeocoderUnavailable, GeocoderTimedOut, GeocoderServiceError
from geopy.geocoders import Nominatim

nhadat24h_df = pd.read_json('rent_data.json')

In [2]:
title = []                  # 1
price = []                  # 2
area = []                   # 3
bedroom = []        # 4
description = []            # 5
width = []                  # 6
direction = []              # 7
toilet = []         # 8
address = []               # 9
geo_code = []               # 10
law = []
floor = []
district = []
province = []
type = []
type_estate = []
image_links = []


In [3]:
# Add new column with default None values for geocodes
nhadat24h_df["geo_code"] = [None for _ in range(len(nhadat24h_df))]

def get_geocode(address):
    geolocator = Nominatim(user_agent="my_geocoder")
    
    # Retry parameters
    max_retries = 3
    retry_delay = 2  # seconds
    
    for attempt in range(max_retries):
        try:    
            location = geolocator.geocode(address + ", Vietnam", timeout=15)  # Add country information
            if location:
                return [location.latitude, location.longitude]
            else:
                # print(f"Error: Address not found for {address}!")
                return None
        except (GeocoderUnavailable, GeocoderTimedOut, GeocoderServiceError) as e:            
            print(f"Geocoder unavailable (attempt {attempt + 1}/{max_retries}). Retrying after {retry_delay} seconds.")
            time.sleep(retry_delay)
            retry_delay *= 2  # Exponential backoff
    return None

def data_normalization(df):
    for i, price in enumerate(df['price']):
        df.at[i, 'price'] = price.replace('Triệu', '').strip()
    
    for i, width in enumerate(df['width']):
        if isinstance(width, str):
            df.at[i, "width"] = width.replace('(m)', '').strip()
        
    for i, floor in enumerate(df['floor']):
        if isinstance(floor, str):
            df.at[i, "floor"] = floor.replace('(Tầng)', '').strip()

data_normalization(nhadat24h_df)

if 'address' in nhadat24h_df.columns:
    for i, address in enumerate(nhadat24h_df["address"]):
        nhadat24h_df.at[i, "geo_code"] = get_geocode(address)
else:
    print("Error: 'address' column not found in the DataFrame.")

def write_data(df, field, data_list):
    if not isinstance(data_list, list):
        data_list = []  # Initialize as empty list if not already
    
    if field in df.columns:
        for value in df[field]:
            data_list.append(value)
    else:
        print(f"Error: '{field}' column not found in the DataFrame.")

write_data(nhadat24h_df, "title", title)
write_data(nhadat24h_df, "price", price)
write_data(nhadat24h_df, "area", area)
write_data(nhadat24h_df, "type", type)
write_data(nhadat24h_df, "type_estate", type_estate)
write_data(nhadat24h_df, "image_links", image_links)
write_data(nhadat24h_df, "bedroom", bedroom)
write_data(nhadat24h_df, "description", description)
write_data(nhadat24h_df, "width", width)
write_data(nhadat24h_df, "direction", direction)
write_data(nhadat24h_df, "toilet", toilet)
write_data(nhadat24h_df, "address", address)
write_data(nhadat24h_df, "law", law)
write_data(nhadat24h_df, "floor", floor)
write_data(nhadat24h_df, "geo_code", geo_code)

# Display the updated DataFrame
print("\nUpdated DataFrame:")


Updated DataFrame:


In [4]:
nhadat24h_df

,title,price,area,type,address,description,law,type_estate,province,district,bedroom,toilet,floor,direction,width,image_links,geo_code
0,Chính chủ cho thuê cả nhà tại 135 khâm thiên đ...,Thỏa thuận,55,Cho Thuê,"Phường Khâm Thiên, Quận Đống Đa, Hà Nội","Chính chủ cho thuê cả nhà tại 135 Khâm Thiên, ...",None,Nhà mặt phố,Hà Nội,Đống Đa,NaN,NaN,None,None,None,https://nhadat24h.net//Upload/User/DangTin/202...,"[21.0192241, 105.8390581]"
1,Căn hộ dịch vụ giá tốt nhất q12 view đẹp gần n...,6.5,25,Cho Thuê,"Phường Tân Chánh Hiệp, Quận 12, Hồ Chí Minh",Xác thực chính chủ ✨ Căn Hộ Dịch Vụ Giá Tốt nh...,None,Căn hộ chung cư,Hồ Chí Minh,12,1.0,1.0,None,None,None,https://nhadat24h.net//Upload/User/DangTin/202...,"[10.8678527, 106.6231544]"
2,Cho thuê căn b0715 chung cư imperia sky garden...,19,105,Cho Thuê,"Imperia Sky Garden, Phường Minh Khai, Quận Hai...",Xác thực chính chủ CHO THUÊ CĂN B0715 CHUNG CƯ...,None,Căn hộ chung cư,Hà Nội,Hai Bà Trưng,3.0,2.0,None,None,None,https://nhadat24h.net//Upload/User/DangTin/202...,"[20.9988446, 105.8665932]"
3,Cho thuê nhà mặt tiền nhìn phía đông nam ra hồ...,120,195,Cho Thuê,"Quảng An, Phường Quảng An, Quận Tây Hồ, Hà Nội",Xác thực chính chủ Chính chủ cần cho thuê Nhà ...,Có Sổ đỏ,Nhà mặt phố,Hà Nội,Tây Hồ,1.0,2.0,4,Đông nam,13,https://nhadat24h.net//Upload/User/DangTin/202...,"[21.0705937, 105.8299492]"
4,Cho thuê văn phòng gems empire tower 201 trườn...,20,50,Cho Thuê,"Phường Khương Đình, Quận Thanh Xuân, Hà Nội",Không gian đẳng cấp – Trung Tâm Quận Đống Đa –...,None,"Văn phòng, TTTM, Cửa hàng, Kiot",Hà Nội,Thanh Xuân,NaN,NaN,None,None,None,https://nhadat24h.net//Upload/User/DangTin/202...,"[20.9899852, 105.8197278]"
5,Khang thành invest - chuyên cho thuê mini hous...,7,35,Cho Thuê,"Phường Tân An, Quận Ninh Kiều, Cần Thơ",KHANG THÀNH Invest - CHUYÊN CHO THUÊ MINI HOUS...,None,Căn hộ chung cư,Cần Thơ,Ninh Kiều,1.0,1.0,3,None,10,https://nhadat24h.net//Upload/User/DangTin/202...,"[10.0318417, 105.785674]"
6,"Cho thuê mặt bằng mặt tiền phạm ngũ lão, gần c...",45,450,Cho Thuê,"Phường An Hòa, Quận Ninh Kiều, Cần Thơ","CHO THUÊ MẶT BẰNG 500M2, MẶT TIỀN 12m Giá: 45 ...",None,"Nhà xưởng, mặt bằng, kho bãi",Cần Thơ,Ninh Kiều,1.0,2.0,1,None,12,https://nhadat24h.net//Upload/User/DangTin/202...,"[10.0459669, 105.7697904]"


In [5]:
nhadat24h_df.to_csv('rent_data.csv', encoding='utf-8', index=False)


In [6]:
import pymysql

# Thiết lập kết nối tới cơ sở dữ liệu MySQL
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='123456',
    database='estate',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

try:
    with connection.cursor() as cursor:
        # Xóa dữ liệu từ cả hai bảng pools_bds và pools_image
        cursor.execute("""
            DELETE pb, pi
            FROM pools_bds pb
            JOIN pools_image pi ON pb.BDS_id = pi.BDS_id
            WHERE pb.kind = 'Cho Thuê' AND pi.content LIKE '%nhadat24h.net%'
        """)
        connection.commit()

finally:
    connection.close()


In [2]:
import csv
import pymysql

# Thiết lập kết nối tới cơ sở dữ liệu MySQL với mã hóa utf8mb4
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='123456',
    database='estate',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

def is_decimal(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

try:
    with connection.cursor() as cursor:
        
        # Đọc dữ liệu từ file CSV và nhập vào MySQL
        with open('buy_data.csv', 'r', encoding='utf-8') as file:
            csv_data = csv.DictReader(file)
            for row in csv_data:
                try:
                    district = row['district']
                    province_city = row['province']
                    
                    # Kiểm tra tồn tại address
                    cursor.execute("SELECT address_id FROM pools_address WHERE district = %s AND province_city = %s", (district, province_city))
                    existing_address = cursor.fetchone()
                    if existing_address:
                        address_id = existing_address['address_id']
                    else:
                        # Nếu không tìm thấy địa chỉ, bỏ qua dòng này
                        print(f"Không tìm thấy địa chỉ cho dòng: {row}")
                        continue
                    
                    # Kiểm tra pools_typeofbds 
                    bds_type = row['type_estate']
                    cursor.execute("SELECT Type_Id FROM pools_typeofbds WHERE TypeName = %s", (bds_type,))
                    existing_type = cursor.fetchone()
                    if existing_type:
                        type_id = existing_type['Type_Id']
                    else:
                        # Nếu không tìm thấy loại bất động sản, bỏ qua dòng này
                        print(f"Không tìm thấy loại bất động sản cho dòng: {row}")
                        continue
                    
                    # Thêm dữ liệu vào bảng pools_bds
                    title = row['title']
                    price = row['price']
                    area = row['area']
                    kind = row['type']
                    law = row['law']
                    bedroom = row['bedroom']
                    floor = row['floor']
                    description = row['description']
                    width = row['width']
                    direction = row['direction']
                    toilet = row['toilet']
                    location = row['address']
                    geocode = row['geo_code']
                    
                    # Handle 'price' column
                    if price.strip().lower() == 'thỏa thuận':
                        price = 0  # or set to a default value, like 0
                    elif not is_decimal(price):
                        price = 0  # or set to a default value

                    # Handle 'width' column
                    if not is_decimal(width):
                        width = 0  # or set to a default value

                    # Handle 'bedroom' column
                    if not bedroom.strip() or bedroom.lower() == 'null':
                        bedroom = 0  # or set to a default value
                       
                    # Handle 'floor' column 
                    if not floor.strip() or floor.lower() == 'null':
                        floor = 0  # or set to a default value

                    # Handle 'toilet' column (if needed)
                    if not is_decimal(toilet):
                        toilet = 0  # or set to a default value

                    # Kiểm tra geocode nếu là None hoặc rỗng, bỏ qua dòng này
                    if not geocode:
                        # print(f"Không có giá trị geocode cho dòng: {row}")
                        continue

                    # Chuyển geo_code thành string nếu nó là list
                    if isinstance(geocode, list):
                        geocode = str(geocode)

                    # Thêm dữ liệu vào bảng pools_bds
                    cursor.execute("""
                        INSERT INTO pools_bds (title, price, area, kind, law, bedroom, floor, address_id, Type_id, description, width, direction, toilet, location, geocode)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """, (title, price, area, kind, law, bedroom, floor, address_id, type_id, description, width, direction, toilet, location, geocode))
                    connection.commit()
                    
                    # Lấy BDS_id của BDS vừa thêm
                    cursor.execute("SELECT LAST_INSERT_ID()")
                    bds_id = cursor.fetchone()['LAST_INSERT_ID()']
                    
                    # Thêm dữ liệu vào bảng pools_image (nếu có)
                    if 'image_links' in row and row['image_links']:  # Kiểm tra xem cột 'image_link' có tồn tại và có giá trị không
                        image_link = row['image_links']
                        cursor.execute("INSERT INTO pools_image (BDS_id, content) VALUES (%s, %s)", (bds_id, image_link))
                        connection.commit()
                        
                except Exception as e:
                    print(f"Lỗi khi xử lý dòng: {row}")
                    print(e)
finally:
    connection.close()
